This is how to wrap a whole SQL query in a condition (such that the query won't happen unless condition is met) using SQL:

In [ ]:
IF (Try_Cast(@GroupId as int) > 0)
BEGIN

SELECT
    p.Id
    , CONCAT(p.NickName, ' ', p.LastName) AS 'Name'
    , CASE gm.GroupMemberStatus
        WHEN 0 THEN 'Inactive'
        WHEN 1 THEN 'Active'
        WHEN 2 THEN 'Pending'
    END AS 'Member Status'
    , CASE p.Gender
        WHEN 1 THEN 'Male'
        WHEN 2 THEN 'Female'
        ELSE 'Unknown'
    END AS 'Gender'
    , gtr.Name AS 'Member Role'
    , dbo.ufnCrm_GetAge(p.Birthdate) AS 'Age'
FROM
    [GroupMember] gm
    INNER JOIN [GroupTypeRole] gtr ON gtr.Id = gm.GroupRoleId
    INNER JOIN [Person] p ON p.Id = gm.PersonId
WHERE
    gm.GroupId = @GroupId
    AND
    gm.IsArchived=0
GROUP BY
    p.Id
    , p.NickName
    , p.LastName
    , gm.GroupMemberStatus
    , p.Gender
    , gtr.Name
    , dbo.ufnCrm_GetAge(p.Birthdate)
    , p.Email
    , gtr.[Order]
ORDER BY
    gtr.[Order]
    , gm.GroupMemberStatus ASC
    , p.LastName

END

In this above example, if the condition in line 1 is FALSE, everything after line 1 pretends not to exist.
If the condition in line 1 is TRUE, then the SQL between BEGIN and END runs.

This is how to wrap a whole SQL query in a condition (such that the query won't happen unless condition is met) using Lava:

In [ ]:
{% assign coachGuid = PageParameter.LGCoach | SanitizeSql %}

{% if coachGuid != empty and coachGuid != null %}
SELECT
    g.Id AS 'GroupId'
    , g.Name AS 'Group Name'
    , COUNT(DISTINCT CASE WHEN gm.IsArchived = 0 AND gm.GroupMemberStatus = 1 THEN gm.Id END) AS 'Active Members'
    , COUNT(DISTINCT CASE WHEN gm.IsArchived = 0 AND gm.GroupMemberStatus = 2 THEN gm.Id END) AS 'Pending Members'
    , c.Name AS 'Campus'
    , av0.[Value] AS 'var1'

FROM
    [Group] g
    INNER JOIN [GroupMember] gm ON gm.GroupId = g.Id
    LEFT JOIN [AttributeValue] av0 ON av0.EntityId = g.Id AND av0.AttributeId IN (8975, 8978, 8982, 8979)
    LEFT JOIN [AttributeValue] av2 ON av2.EntityId = g.Id AND av2.AttributeId = 8975 --Coach
    LEFT JOIN [Campus] c ON c.Id = g.CampusId
WHERE
    g.GroupTypeId IN (25, 45, 46, 59) AND g.IsActive = 1 AND g.IsArchived = 0
    AND
    av0.AttributeId = 8975
    AND
    av2.[Value] = '{{ coachGuid }}'
GROUP BY
    g.Id
    , g.Name
    , c.Name
    , av0.[Value]
ORDER BY
    g.Name
{% endif %}

In this above example, if the condition in line 3 is FALSE, everything after line 3 pretends not to exist.
If the condition in line 3 is TRUE, then the SQL between {% if %} and {% endif %} runs.

It's worth mentioning that, if the context where I'm writing this code is a SQL engine, the above will work fine;
whereas if the context where I'm writing this code is a Lava engine, I will have to wrap the SQL inside {% sql %} and {% endsql %}